In [17]:
import $file.hw9Bstdlib
import $file.hw9Bparserlib
import hw9Bstdlib._
import hw9Bparserlib._

SyntaxError: invalid syntax (<ipython-input-17-d9359f2ff259>, line 1)

# Homework 9B - The Lettuce Parser

We have an eval function from hw7 and a lexer from hw9. Once we create our parser we can hook everything together to have a functioning language.

## The Interpreter
Below is the solution to the full eval function from hw7

In [1]:
//
// Expressions
//
sealed trait Expr
case class Num(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr
case class FunDef(p: String, body: Expr) extends Expr
case class FunCall(e_func: Expr, e_arg: Expr) extends Expr
case class LetRec(f_name: String, arg_name: String, e_body: Expr, e_in: Expr) extends Expr

//
// Values
//
sealed trait Value
case class NumVal(x : Integer) extends Value
case class BinVal(x : Bool) extends Value
case object Error extends Value
case class Closure(p: String, body: Expr, env: Environment) extends Value

//
// Environment
//
sealed trait Environment
case object EmptyEnv extends Environment 
case class Extend(k : String, v : Value, env : Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment
def lookupEnv(sigma: Environment, x: String): Maybe[Value] = sigma match {
    case EmptyEnv => None
    case Extend(y, v, pi) => string_eq(y,x) match {
        case True => Just(v)
        case False => lookupEnv(pi, x)
    } 
    case ExtendRec(f, y, e, pi) => string_eq(x,f) match {
        case True => Just(Closure(y, e, sigma))
        case False => lookupEnv(pi, x)
    } 
}

//
// Eval
//
def eval(env : Environment, expr : Expr) : Value = expr match {
    case Num(n)    => NumVal(n)
    case Bin(p)    => BinVal(p)
    case Ident(id) => lookupEnv(env, id) match {
        case Just(v) => v
        case None    => Error
    }
    case Plus(e1, e2)  => eval_bin_arith(plus, env, e1, e2)
    case Minus(e1, e2) => eval_bin_arith(minus, env, e1, e2)
    case Mult(e1, e2)  => eval_bin_arith(mult, env, e1, e2)
    case Pow(e1, e2)   => (eval(env, e1), eval(env,e2)) match {
        case (NumVal(x), NumVal(Positive(n))) => NumVal(pow(x, n))
        case _ => Error
    }
    case Neg(e) => eval(env, e) match {
        case NumVal(x) => NumVal(negate(x))
        case BinVal(p) => BinVal(not(p))
        case Error     => Error
    }
    case Eq(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (NumVal(x), NumVal(y)) => BinVal(int_eq(x,y))
        case (BinVal(p), BinVal(q)) => BinVal(bool_eq(p,q))
        case _                      => Error
    }
    case And(e1, e2) => eval_bin_bool(and, env, e1, e2)
    case Or(e1, e2)  => eval_bin_bool(or, env, e1, e2)
    case IfThenElse(p, e_t, e_f) => eval(env, p) match{
        case BinVal(True)  => eval(env, e_t)
        case BinVal(False) => eval(env, e_f)
        case _             => Error
    }
    case Let(id, df, body) => eval(env, df) match{
        case Error => Error
        case x     => {
            val new_env = Extend(id, x, env)
            eval(new_env , body)
        }
    }
    
    case FunDef(x, e_body) =>
        Closure(x, e_body, env)
    
    case FunCall(e_f, e_arg) =>
        eval(env, e_f) match {
            case Closure(p, e_body, pi) => eval(env, e_arg) match {
                case Error => Error
                case v_a   => eval(Extend(p, v_a, pi), e_body)
            }
            case _ => Error
        }
    
    case LetRec(x_name, x_param, e_body, e_in) =>
        eval(ExtendRec(x_name, x_param, e_body, env), e_in)
}

def eval_bin_arith( op : (Integer, Integer) => Integer
                  , env : Environment
                  , e1 : Expr
                  , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (NumVal(x), NumVal(y)) => NumVal(op(x,y))
        case _ => Error
    }

def eval_bin_bool( op : (Bool, Bool) => Bool
                 , env : Environment
                 , e1 : Expr
                 , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (BinVal(x), BinVal(y)) => BinVal(op(x,y))
        case _ => Error
    }

SyntaxError: invalid syntax (<ipython-input-1-58016d1608a6>, line 1)

## The Lexer

Below is the solution to the lexer from hw9.

In [0]:
sealed trait KW
case object KWFunction extends KW
case object KWLet extends KW
case object KWIn extends KW
case object KWRec extends KW
case object KWIf extends KW
case object KWThen extends KW
case object KWElse extends KW

sealed trait Lit
case class NumLiteral(x : Nat) extends Lit
case class BoolLiteral(x : Bool) extends Lit

sealed trait Lex 
case class Identifier(s : List[Char]) extends Lex
case class Keyword(k : KW) extends Lex
case class Operator(s : List[Char]) extends Lex
case class Literal(l : Lit) extends Lex
case class Comment(s : List[Char]) extends Lex 
case object OpenParen extends Lex
case object CloseParen extends Lex

def const[S,D,E](p : Parser[S,D], d : E) : Parser[S, E] = bind(p, (x : D) => success(d))

def pLet : Parser[Char, Lex] = const(string("let"), Keyword(KWLet))
def pFunc : Parser[Char, Lex] = const(string("function"), Keyword(KWFunction))
def pRec : Parser[Char, Lex] = const(string("rec"), Keyword(KWRec))
def pIn : Parser[Char, Lex] = const(string("in"), Keyword(KWIn))
def pIf : Parser[Char, Lex] = const(string("if"), Keyword(KWIf))
def pThen : Parser[Char, Lex] = const(string("then"), Keyword(KWThen))
def pElse : Parser[Char, Lex] = const(string("else"), Keyword(KWElse))
def keywords : Parser[Char, Lex] = option(pLet, 
                                   option(pFunc, 
                                   option(pRec, 
                                   option(pIn,
                                   option(pIf, 
                                   option(pThen, pElse))))))

def pTrue : Parser[Char, Lex] = const(string("true"), Literal(BoolLiteral(True)))
def pFalse : Parser[Char, Lex] = const(string("false"), Literal(BoolLiteral(False)))
def pNumber : Parser[Char, Lex] = bind(number, 
                                        (n : Nat) => success(Literal(NumLiteral(n))))
def literals : Parser[Char, Lex] = option(pTrue, 
                                   option(pFalse, pNumber))

def isAlpha(c : Char) : Bool = if(c >= 'A' && c <= 'Z' || c >= 'a' && c <= 'z') True else False
def alphanum : Parser[Char, Char] = option(digit, satisfies(isAlpha))
def identifier : Parser[Char, Lex] = bind(satisfies(isAlpha), 
                                          (x : Char) => bind(many(alphanum),
                                          (xs : List[Char]) => success(Identifier(Cons(x,xs)))))

def isOperator(x : Char) : Bool = x match {
    case '+' => True
    case '-' => True
    case '*' => True
    case '^' => True
    case '=' => True
    case '&' => True
    case '|' => True
    case _   => False
}
def operators : Parser[Char, Lex] = option(
                    bind(string("=="),          (str : List[Char]) => success(Operator(str))),
                    bind(satisfies(isOperator), (c : Char) => success(Operator(singleton(c)))))

def parens : Parser[Char, Lex] = option(
                                    const(char('('), OpenParen) ,
                                    const(char(')'), CloseParen))

def lexOne : Parser[Char, Lex] = option(keywords, 
                                 option(operators, 
                                 option(parens,
                                 option(literals,
                                        identifier))))

def lexWhitespace : Parser[Char, List[Lex]] = const(whitespace, Empty)

def lexParens : Parser[Char, List[Lex]] = bind(parens, (x : Lex) => 
                                          bind(maybeWhitespace,
                                               (ws : List[Char]) => success(singleton(x))))


def lexToken : Parser[Char, List[Lex]] = bind(lexOne, 
                                             (l : Lex) => bind(option(lexParens, lexWhitespace),
                                             (sep : List[Lex]) => success(Cons(l, sep))))

def concat[A](xs : List[List[A]]) : List[A] = xs match {
    case Empty => Empty
    case Cons(xs, xxs) => append(xs, concat(xxs))
}


def lexer : Parser[Char, List[Lex]] = bind(many(lexToken), 
                   (ls : List[List[Lex]]) =>
                   success(concat(ls)))

## Formal Grammars

While we have been using BNF grammars to represent constructed values, their original purpose was to define languages.
The below grammar defines the language with only the strings `"ab"` and `bc`
$$
\begin{align}
    S ::=&\ \texttt{a}\ \texttt{b} \\
     \mid&\ \texttt{b}\ \texttt{c}
\end{align}
$$

Each production (each "thing" seperated by $\mid$ operators to the right of the $::=$) is read verbatim other than the non terminals. Non-terminals are the names defined to the left of the $::=$. In this case the only non-terminal is $S$. Non-terminals are expanded recursively.

The equivalent parser would be:
```scala
val S = choose(
    bind(char('a'), (_: Char) => char('b')),
    bind(char('b'), (_: Char) => char('c'))
)
```

A recursive example could be something like:
$$
\begin{align}
    S ::=&\ \texttt{a}\ S\ \texttt{b} \\
     \mid&\ \texttt{z}\ \texttt{X} \\
     \mid&\ \epsilon
\end{align}
$$
This definition also includes the special symbol $\epsilon$ (epsilon). This means the empty string.

Examples of strings in this language are:
```
"ab"
"aaabbb"
"aaazXbbb"
"zX"
""
```

And the equivalent scala parser would be:
```scala
val S = choose(
    bind(char('a'), (_: Char) => bind(S, (_: Char) => char('b'))), // a S b
    choose(
        bind(char('z'), (_: Char) => char('X')),                   // z X
        success(???)                                               // epsilon
    )
)
```

### A basic lettuce grammar
To write the parser for Lettuce, we first have to give a formal specification of the language.
This is given below:

$$
\begin{align}
    Lettuce ::=&\ E\\
    \\
    Z ::=&\ ... &\text{Imaginary definition for numbers}\\
    \\
    Ident ::=&\ ... &\text{Imaginary definition for variables}\\
     \\
    B ::=&\ \texttt{true}\\
      \mid&\ \texttt{false}\\
     \\
    E ::=&\ \color{green}{Z} &\text{Numbers}\\
     \mid&\ \color{green}{B} &\text{Booleans}\\
     \mid&\ \color{green}{Ident} &\text{Variable names}\\
     \mid&\ \color{green}{E \texttt{ + } E}  &\text{Addition}\\
     \mid&\ \color{green}{E \texttt{ * } E}  &\text{Multiplication}\\
     \mid&\ \color{green}{\texttt{-} E}  &\text{Numeric Negation}\\
     \mid&\ \color{green}{\texttt{( } E \texttt{ )}}  &\text{Parentheses}\\
     \mid&\ \color{green}{E \texttt{ ( } E \texttt{ )}}  &\text{Function Calls}\\
     \mid&\ \color{green}{\texttt{if } E \texttt{ then } E \texttt{ else } E}  &\text{If-Else}\\
     \mid&\ \color{green}{\texttt{function ( } Ident \texttt{ ) } E}  &\text{Function Definition}\\
     \mid&\ \color{green}{\texttt{let } Ident \texttt{ = } E \texttt{ in } E}  &\text{Let bindings}\\
     \mid&\ E \texttt{ - } E  &\text{Subtraction}\\
     \mid&\ E \texttt{ ^ } E  &\text{Exponentiation}\\
     \mid&\ E \texttt{ && } E  &\text{And}\\
     \mid&\ E \mid \mid E  &\text{Or}\\
     \mid&\ E \texttt{ == } E &\text{Equality} \\
     \mid&\ \texttt{!} E  &\text{Boolean Negation}\\
     \mid&\ \texttt{let rec } Ident \texttt{ = } \texttt{function ( } Ident \texttt{ ) } E \texttt{ in } E  &\text{Recursive let bindings}\\
      \\
\end{align}
$$

We can imagine that we have a definition for integers ($Z$) and variable names ($Ident$) because they are overly complicated to write out correctly.

The words to the right side are just an english explanation of each rule, they are not part of the actual grammar. You can see above that we have defined Lettuce as an expression represented by the $E$ *non-terminal*. Just like how we've used BNF before, anything to the left of a $::=$ definition is a non-terminal, which means it can expand to any of the *productions* on the righthand side, each seperated with a pipe operator.

**Note**: Because this would be a huge parser if we wrote all of it (and many of the rules are very similar to parse), we will instead focus on only the $\color{green}{\textbf{green}}$ subset.

### Using the grammar

A valid lettuce progrram is defined as one where we can write it using only the above productions, and which has no more non-terminals in it.

For example, the following program: `let x = 5 in x + x` can be created by applying the rules in the following order (we always start with the $Lettuce$ non terminal when writing a lettuce program):

$$
\begin{align}
    & Lettuce \\
    & E \\
    & \texttt{let } Ident \texttt{ = } E \texttt{ in } E \\
    & \texttt{let x = } E \texttt{ in } E \\
    & \texttt{let x = } Z \texttt{ in } E \\
    & \texttt{let x = 5 in } E \\
    & \texttt{let x = 5 in } E \texttt{ + } E \\
    & \texttt{let x = 5 in } Ident \texttt{ + } E \\
    & \texttt{let x = 5 in x + } E \\
    & \texttt{let x = 5 in x + } Ident \\
    & \texttt{let x = 5 in x + x} \\
    \\
\end{align}
$$

### Problems with this naive approach

There are some problems with the above definition:
1. It is *left recursive*, which means that there is at least one production that has the non-terminal as the versy first thing in the production with no terminals (anything that isn't expanded) before it. An example of such a production is the addition rule, which has $E$ as the very first element. The problem with this is that no characters are consumed before we recurse, so the parsing could continue to loop indefinitely until we get a stack overflow.
2. It doesn't enforce *operator associativity*. Meaning, `1 + 2 + 3` could parse as either `(1 + 2) + 3`, or `1 + (2 + 3)` In this case, the problem is that the same program could have different results for different runs, which would be awful for the user of our language to debug.
3. It doesn't enforce *operator precedance*, meaning `1 + 2 * 3` could parse as `(1 + 2) * 3`, instead of the desired `1 + (2 * 3)`

To solve all problems at once, we redefine the grammar to get rid of left recursion, while structuring it in a way that operators have the desired associativity and precedance. We do this by introducing levels of expressions, that let the nested expressions change to a stronger precedence, but not a weaker one. Here is an example for just plus and multiplication:

**Note**: $\epsilon$ below is a special non-terminal that signifies the empty string.

$$
\begin{align}
    E0 ::=&\ E1\ Pluses \\
    Pluses ::=&\ \texttt{+ } E1\ Pluses \\
          \mid&\ \epsilon \\
    \\
    E1 ::=&\ E2\ Mults \\
    Mults ::=&\ \texttt{* } E2\ Mults \\
         \mid&\ \epsilon \\
    \\
    E2 ::=&\ Z \\
      \mid&\ \texttt{( } E0 \texttt{ )} \\
\end{align}
$$

1. You can see that there is no more left recursion in the grammar.
2. By using $Pluses$ to parse addition left to right, we can control the associativity of the operator.
3. By defining addition to only have $E1$ as a nested expression, we can control the precedence of the plus operator.

We can still multiply the result of additions of course, but now we must do it by parenthesizing the addition, just like in math.

###  Fixed grammar

**Note**: $E^0, E^1, ...$ are the non-terminals that represent different precedence levels. The superscript isn't a special notation, it's just easier to read than $E0, E1, ...$.

$$
\begin{align}
    E ::=&\ E^0 \\
    \\
    E^0 ::=&\ E^1 Pluses \\
    Pluses ::=&\ \texttt{+ } E^1 Pluses \\
          \mid&\ \epsilon \\
    \\
    E^1 ::=&\ E^2 Mults \\
    Mults ::=&\ \texttt{* } E^2 Mults \\
         \mid&\ \epsilon \\
    \\
    E^2 ::=&\ - E^2 \\
       \mid&\ E^3 \\
    \\
    E^3 ::=&\ E^4 Calls \\
    Calls ::=&\ \texttt{( } E^4 \texttt{ ) } Calls \\
         \mid&\ \epsilon \\
    \\
    E^4 ::=&\ Z \\
       \mid&\ B \\
       \mid&\ Ident \\
       \mid&\ \texttt{let } Ident \texttt{ = } E \texttt{ in } E \\
       \mid&\ \texttt{function ( } Ident \texttt{ ) } E \\
       \mid&\ \texttt{if } E \texttt{ then } E \texttt{ else } E \\
       \mid&\ \texttt{( } E \texttt{ )} \\
    \\
\end{align}
$$

### New library funcs

We have defined a new helper in the next cell that you may find useful.

In [0]:
def list_to_string(l: List[Char]): String =
    fold((_: Char) + (_: String), "", reverse(l))

//
// Ignore Below
//

// Translates between built in booleans and our Bool type
def to_bool(b: Boolean): Bool = if (b) True else False

// Gets the data portion out of parsed pair
def get_data[S, D](parse: (D, List[S])): D = parse._1

def parse_finished[A, B](parse: (A, List[B])): Bool = to_bool(parse._2 == Empty)

def lexEq(l1: Lex, l2: Lex): Bool = if (l1 == l2) True else False

### Running the parser
The new parser you're going to write will need to be run on the result of the lexer, so we have `runExprParser` defined below to make this easier.

In [0]:
def runExprParser(p : Parser[Lex, Expr], s : String) : Maybe[Expr] = runParser(lexer, s) match {
    case Just(l) => map(get_data[Lex, Expr], filter(parse_finished, p(l))) match {
        case Cons(expr, _) => Just(expr)
        case _ => None
    } 
    case None => None
}

### New Parser combinators
Just as we had some primitives to build up parsers for `Char`s, we built new ones for dealing with lex tokens as our symbols. With characters, we only had the `char` function to generate parsers, but `Lex` values have more structure, so we now have the following: 

* `def parseOperator(op_str: String)` - This parser generator takes a string and produces a parser that expects that operator. Example:
    ```
    parseOperator("==") // Accepts only `Operator("==")`
    ```

* `def parseNumLex` and `def parseBoolLex` - Take no argumenst, parse a `Literal(NumLiteral(...))` and `Literal(BoolLiteral(...))` lexeme into a `Nat` and a `Bool` respectively.
* `def parseKeyword(kw : KW)` - Parses the given keyword. Example:
    ```
    parseKeyword(KWLet) // Accepts only `KWLet` from our list of lexemes
    ```
* `def parseOpenParen` and `def parseCloseParen` - Parses the given keyword
    ```
    parseKeyword(KWLet) // Accepts only `KWLet` from our list of lexemes
    ```
* `def parseIdentifierLex` - Parses the `Identifier` lexeme into the string representing the variable name. 

In [0]:
def parseNumLex : Parser[Lex, Nat] = (l : List[Lex]) => l match {
    case Cons(Literal(NumLiteral(n)), xs) => singleton((n, xs))
    case _ => Empty
}

def parseBoolLex : Parser[Lex, Bool] = (l : List[Lex]) => l match {
    case Cons(Literal(BoolLiteral(n)), xs) => singleton((n, xs))
    case _ => Empty
}

def parseOperator(op_str: String) : Parser[Lex, Lex] = satisfies(lexEq(_, Operator(string_to_list(op_str))))

def parseKeyword(kw : KW) : Parser[Lex, KW] = (ls : List[Lex]) => ls match {
    case Cons(Keyword(`kw`), xs) => singleton(kw, xs)
    case _ => Empty
}

def parseIdentifierLex : Parser[Lex, List[Char]] = (ls : List[Lex]) => ls match {
    case Cons(Identifier(x), xs) => singleton(x, xs)
    case _                       => Empty
}


def parseOpenParen : Parser[Lex, Lex] = (ls : List[Lex]) => ls match {
    case Cons(OpenParen, xs) => singleton(OpenParen, xs)
    case _                   => Empty
}

def parseCloseParen : Parser[Lex, Lex] = (ls : List[Lex]) => ls match {
    case Cons(CloseParen, xs) => singleton(CloseParen, xs)
    case _                    => Empty
}

## Problem: Parser (40pts)
Define the parser given by the fixed grammar above by filling in the code bellow. Because the entire parser is mutually recursive, all code must unfortunately be compiled in the same cell. Which can get annoying since there are many `???`s. To get around this, you'll want to coment out or stub out sections to test subsets as you implement them. For example, until you are ready to implement multiplication, pass over `parseE1` by setting it to the following (though you should leave a `// TODO` comment so you don't forget to fill it in later).

```scala
def parseE1: Parser[Lex, Expr] =
    parseE2 // Do nothing, move on to E2 parser
```

The suggested order of attack is following the tests:
```
parseBoolLit
parseIdentifier
parseE1 && parseMults
parseE2
parseE3 && parseCalls
parseLet
parseIf
parseParenthesized
parseFunction
```

In [16]:
//
// E
//
def parseE: Parser[Lex, Expr] =
    parseE0(_) // ignore the `(_)`

//
// E^0
//
def parseE0: Parser[Lex, Expr] =
    bind(parseE1, parsePluses)

def parsePluses(e1: Expr): Parser[Lex, Expr] =
    option(bind(parseOperator("+"), (_: Lex) =>
           bind(parseE1,            (e2: Expr) =>
                parsePluses(Plus(e1, e2)))),
           success(e1))

//
// E^1
//
def parseE1: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

def parseMults(e1: Expr): Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

//
// E^2
//
def parseE2: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

//
// E^3
//
def parseE3: Parser[Lex, Expr] =
    bind(parseE4, parseCalls)
def parseCalls(e1: Expr): Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

//
// E^4
//
def parseE4: Parser[Lex, Expr] =
    choose(parseBoolLit,
    choose(parseNumLit,
    choose(parseIdentifier,
    choose(parseLet,
    choose(parseFunction,
    choose(parseIf,
           parseParenthesized))))))


def parseNumLit: Parser[Lex, Expr] =
    bind(parseNumLex, (n: Nat) =>
         success(Num(Positive(n))))

def parseBoolLit: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

def parseIdentifier: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

def parseLet: Parser[Lex, Expr] =
    bind(parseKeyword(KWLet), (_: KW) =>
    bind(parseIdentifierLex,  (x : List[Char]) =>
    bind(parseOperator("="),  (_: Lex) =>
    bind(parseE,              (e1 : Expr) =>
    bind(parseKeyword(KWIn),  (_ : KW) =>
    bind(parseE,              (e2 : Expr) =>
         success(Let(list_to_string(x), e1, e2))))))))

def parseIf: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

def parseParenthesized: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

def parseFunction: Parser[Lex, Expr] =
    // YOUR CODE HERE
    ???

SyntaxError: invalid syntax (<ipython-input-16-6ec9d6e13633>, line 1)

In [0]:
def interpret(program: String): Maybe[Value] = runExprParser(parseE, program) match {
    case Just(e) => Just(eval(EmptyEnv, e))
    case None => None
}

In [0]:
// Booleans
assert(runExprParser(parseE, "true ") == Just(Bin(True)))
assert(runExprParser(parseE, "false ") == Just(Bin(False)))
passed(3)

In [0]:
// Identifiers
assert(runExprParser(parseE, "x ") == Just(Ident("x")))
assert(runExprParser(parseE, "abcd ") == Just(Ident("abcd")))
passed(4)

In [0]:
// Multiplication
val x = Ident("x")
assert(runExprParser(parseE, "x * x ") == Just(Mult(x, x)))
assert(runExprParser(parseE, "x * x * y ") == Just(Mult(Mult(x, x), Ident("y"))))
assert(runExprParser(parseE, "x + x * y ") == Just(Plus(x, Mult(x, Ident("y")))))
passed(6)

In [0]:
// Negation
val x = Ident("x")
assert(runExprParser(parseE, "- x ") == Just(Neg(x)))
assert(runExprParser(parseE, "- - - x ") == Just(Neg(Neg(Neg(x)))))
assert(runExprParser(parseE, "- x * - x ") == Just(Mult(Neg(x), Neg(x))))
passed(5)

In [0]:
// Function Calls
val f = Ident("f")
val x = Ident("x")
assert(runExprParser(parseE, "f ( x ) ") == Just(FunCall(f, x)))
assert(runExprParser(parseE, "f ( x ) ( x ) ") == Just(FunCall(FunCall(f, x), x)))
assert(runExprParser(parseE, "- f ( x ) ") == Just(Neg(FunCall(f, x))))
passed(6)

In [0]:
// Parens
val x = Ident("x")
assert(runExprParser(parseE, "( x ) ") == Just(x))
assert(runExprParser(parseE, "( ( x ) ) ") == Just(x))
assert(runExprParser(parseE, "( x + x ) * x ") == Just(Mult(Plus(x, x), x)))
passed(5)

In [0]:
// If
val x = Ident("x")
val y = Ident("y")
assert(runExprParser(parseE, "if x then x else x ") == Just(IfThenElse(x, x, x)))
assert(runExprParser(parseE, "y + if x then y else ( x + y ) ") == Just(Plus(y, IfThenElse(x, y, Plus(x, y)))))
assert(runExprParser(parseE, "if if x then x else x then if x then x else x else if x then x else x ")
       == Just(IfThenElse(IfThenElse(x, x, x), IfThenElse(x, x, x), IfThenElse(x, x, x))))
passed(3)

In [0]:
// Functions
val x = Ident("x")
val y = Ident("y")
assert(runExprParser(parseE, "function ( x ) x ") == Just(FunDef("x", x)))
assert(runExprParser(parseE, "( function ( x ) x ) ( y ) ") == Just(FunCall(FunDef("x", x), y)))
assert(runExprParser(parseE, "function ( x + x ) x ") == None)
passed(5)